<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Lab: Cleaning Rock Song Data

_Authors: Dave Yerrington (SF)_

---


In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import sqlite3

%matplotlib inline

### 1. Load `rock.csv` and do an initial examination of its data columns.

In [2]:
rockfile = "./datasets/rock.csv"

In [3]:
# Load the data.
rock = pd.read_csv(rockfile)

In [4]:
# Look at the information regarding its columns.
rock.columns

Index([u'Song Clean', u'ARTIST CLEAN', u'Release Year', u'COMBINED', u'First?',
       u'Year?', u'PlayCount', u'F*G'],
      dtype='object')

### 2.  Clean up the column names.

Let's clean up the column names. There are two ways we can accomplish this:

#### 2.A Change the column names when you import the data using `pd.read_csv()`.

Notice that, when passing `names=[..A LIST OF STRING..]` with a number of columns that matches the number of strings in the passed list, you replace the column names.

NOTE: When you create custom column names, the first row of the `.csv` already represents a header. It is important to tell `pandas` to skip that row. The `skiprows=1` keyword argument to `read_csv()` will tell `pandas` to skip the first row.

In [5]:
# Change the column names when loading the '.csv':
rock2 = pd.read_csv(rockfile, names = ['Song Clean','Artist Clean','Release Year','Combined','First','Year','PlayCount','FG'], skiprows=1)
rock2.head()

,Song Clean,Artist Clean,Release Year,Combined,First,Year,PlayCount,FG
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


#### 2.B Change column names using the `.rename()` function.

The `.rename()` function takes an argument, `columns=name_dict`, in which `name_dict` is a dictionary containing the original column names as keys and the new column names as values.

In [6]:
# Change the column names using the `.rename()` function.
name_dict = {'Song Clean': 'Song Clean', 'ARTIST CLEAN':'Artist Clean', 'Release Year':'Release Year',\
             'COMBINED':'Combined','First?': 'First','Year?':'Year','PlayCount':'Play Count', 'F*G': 'FG'}
rock3 = rock.rename(columns = name_dict)
rock3.head()

,Song Clean,Artist Clean,Release Year,Combined,First,Year,Play Count,FG
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


#### 2.C Reassigning the `.columns` attribute of a DataFrame.

You can also just reassign the `.columns` attribute to a list of strings containing the new column names. 

The only caveat with reassigning `.columns` is that you have to reassign all of the column names at once. You can't partially replace a value by working on `.columns` directly. You have to reassign `.columns` with a list of equal length. 

In [7]:
# Replace the column names by reassigning the `.columns` attribute.

rock.columns = ['Song Clean','Artist Clean','Release Year','Combined','First','Year','PlayCount','FG']
rock.columns

Index([u'Song Clean', u'Artist Clean', u'Release Year', u'Combined', u'First',
       u'Year', u'PlayCount', u'FG'],
      dtype='object')

### 3. Subsetting data where null values exist.

We have mixed `str` and `NaN` values in the `release` column. `NaN` stands for "not a number" and is the way `pandas` handles "nulls" or nonexistent data. We can use the `.isnull()` method of a Series to find null values.

Print the header of the data subset to where the `release` column is null values.

In [8]:
# Show records where df['release'] is null
asd = pd.isnull(rock['Release Year']) 

rock[asd][0:10][['Release Year']]

,Release Year
1,NaN
10,NaN
13,NaN
16,NaN
24,NaN
25,NaN
26,NaN
29,NaN
31,NaN
35,NaN


### 4. Update slices of your DataFrame based on mask selection/slices.

In many scenarios, we want to upate values in our DataFrame according to criteria. Let's say we wanted to set all of the null values in `release` to 0.

With newer versions of `pandas`, in order to manipulate data in the original DataFrame, we have to use `.loc` while performing reassignment using a mask and an index.

For example, the following won't always work:
```python
df[row_mask]['column_name'] = new_value
```

The best way to accomplish the same task is:
```python
df.loc[row_mask, 'column_name'] = new_value
```

For multiple column assignment, you would use:
```python
df.loc[row_mask, ['col_1', 'col_2', 'col_3']] = new_value
```

#### 4.A Let's try it out. Make all of the null values in `release` 0.

In [9]:
# Replace release nulls with 0
rock.loc[asd, 'Release Year'] = 0

#### 4.B Verify that `release` contains no null values.

In [10]:
# A:
check_null = pd.isnull(rock['Release Year'])
rock[check_null]

,Song Clean,Artist Clean,Release Year,Combined,First,Year,PlayCount,FG


### 5. Ensure that the data types of the columns make sense. 

Verifying column data types is a critical part of data munging. If columns have the wrong data type, then there is usually corrupted or incorrect data in some of the observations.

#### 5.A Look at the data types for the columns. Are any incorrect given what the data represents?

In [11]:
# A:
rock.dtypes

Song Clean      object
Artist Clean    object
Release Year    object
Combined        object
First            int64
Year             int64
PlayCount        int64
FG               int64
dtype: object

### 6. Investigate and clean up the `release` column.

The `release` column is a string data type when it should be an integer.

#### 6.A Figure out what value(s) are causing the `release` column to be encoded as a string instead of an integer.

In [12]:
# A:
rock['Release Year'].unique()

array(['1982', 0, '1981', '1980', '1975', '2000', '2002', '1992', '1985',
       '1993', '1976', '1995', '1979', '1984', '1977', '1990', '1986',
       '1974', '2014', '1987', '1973', '2001', '1989', '1997', '1971',
       '1972', '1994', '1970', '1966', '1965', '1983', '1955', '1978',
       '1969', '1999', '1968', '1988', '1962', '2007', '1967', '1958',
       '1071', '1996', '1991', '2005', '2011', '2004', '2012', '2003',
       '1998', '2008', '1964', '2013', '2006', 'SONGFACTS.COM', '1963',
       '1961'], dtype=object)

#### 6.B Look at the rows in which there is incorrect data in the `release` column.

In [13]:
# A:
songfacts = rock['Release Year'] == 'SONGFACTS.COM'
rock[songfacts]

,Song Clean,Artist Clean,Release Year,Combined,First,Year,PlayCount,FG
1504,Bullfrog Blues,Rory Gallagher,SONGFACTS.COM,Bullfrog Blues by Rory Gallagher,1,1,1,1


#### 6.C. Clean up the data. Normally we may replace the offending data with null np.nan values, however we previously converted all of the nan values in the release column to zeros so we might as well continue with the same practice. Replacing with 0 (or nan) will allow us to convert the column to numeric.

In [14]:
# A:
rock.loc[1504, 'Release Year'] = 0

In [15]:
rock.loc[1504]

Song Clean                        Bullfrog Blues
Artist Clean                      Rory Gallagher
Release Year                                   0
Combined        Bullfrog Blues by Rory Gallagher
First                                          1
Year                                           1
PlayCount                                      1
FG                                             1
Name: 1504, dtype: object

In [16]:
rock['Release Year'] = rock['Release Year'].astype(int)

In [17]:
rock['Release Year'].unique()
rock.dtypes

Song Clean      object
Artist Clean    object
Release Year     int64
Combined        object
First            int64
Year             int64
PlayCount        int64
FG               int64
dtype: object

### 7. Get summary statistics for the `release` column using the `.describe()` function.

Now that the `release` column is finally a numeric data type, we can apply the `.describe()` function.  

#### 7.A Print out the summary stats for the `release` column. What is the earliest and latest release date?

In [18]:
# A:
rock['Release Year'].describe()

count    2230.000000
mean     1465.331390
std       867.196161
min         0.000000
25%         0.000000
50%      1973.000000
75%      1981.000000
max      2014.000000
Name: Release Year, dtype: float64

In [19]:
release_year_sort = sorted(rock['Release Year'])
len(release_year_sort)
sum(release_year_sort)/len(release_year_sort)

1465

#### 7.B Based on the summary statistics, is there anything else wrong with the `release` column? 

In [23]:
# A:
#Minimum year is 1071

_Looking at the DataFrame that contains the year 1071, we can see that the year was probably corrupted and should be replaced with something else if possible._

In [24]:
rock[rock['Release Year'] == 1071]

,Song Clean,Artist Clean,Release Year,Combined,First,Year,PlayCount,FG
547,Levon,Elton John,1071,Levon by Elton John,1,1,8,8


In [25]:
rock.loc[547, 'Release Year'] = 1973

In [26]:
rock.loc[547]

Song Clean                    Levon
Artist Clean             Elton John
Release Year                   1973
Combined        Levon by Elton John
First                             1
Year                              1
PlayCount                         8
FG                                8
Name: 547, dtype: object

In [51]:
rock.123 > 1

SyntaxError: invalid syntax (<ipython-input-51-70a46e07b6d3>, line 1)

### 8. Make changes and investigate using custom functions with `.apply()`.

Let's say we want to traverse every single row in our data set and apply a function to that row.

#### 8.A Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970.


In [ ]:
# A:
def func(df):
    

#### 8.B Using the `.apply()` function, apply the function you wrote to the first four rows of the DataFrame.

You will need to tell the `apply` function to operate row by row. Setting the keyword argument as `axis=1` indicates that the function should be applied to each row individually.

In [ ]:
# A:

You'll notice that there will be a final output Series of `None` values. The `.apply()` function, if a return value is not specified, will return a Series of `None` values (similar to how the default return for Python functions is `None` when a return statement is not specified).

### 9. Write a function that converts cells in a DataFrame to float and otherwise replaces them with `np.nan`.

If applied to our data, it would keep only the numeric information and otherwise input null values.

Recall that the try-except syntax in Python is a great way to try something and take another action if the initial step fails:

```python
try:
    Perform some action.
except:
   Perform some other action if the first failed with an error.
```

#### 9.A Write the function that takes a column and converts all of its values to float if possible and `np.nan` otherwise. The return value should be the converted Series.

In [ ]:
# A:

#### 9.B Try your function out on the rock song data and ensure the output is what you expected.


In [ ]:
# A:

#### 9.C Describe the new float-only DataFrame.

In [ ]:
# A: